In [176]:
import pandas as pd
import seaborn as sns

In [177]:
df = pd.read_csv('data_clean/train.csv')
pd.options.display.max_columns = None
df

,TransactionNumber,UserID,Age,Gender,Occupation,EducationLevel,MaritalStatus,NumDependents,Income,Expenditure,GiftsTransaction,TransactionDate,TransactionTime,TransactionAmount,MerchantID,TransactionType,TransactionLocation,DeviceType,Latitude,Longitude,EmailDomain,Terrorism,UserTenure,IsFraud,ExpenditureCurrency,GiftsCurrency,TransactionDatetime,TransactionCurrency
0,8765,70,37,Female,Professional,Bachelor,Widowed,3,28884.43,14610.61,1050.01,2023-03-12,12:25:57,258.14,M006,Withdrawal,Adelaide,Mobile,-31.840233,145.612793,jon44@disposable.com,False,113,1,AUD,GBP,2023-03-12 12:25:57,AUD
1,9645,3386,34,Male,Student,High School,Married,4,54919.07,39169.49,4969.71,2023-03-05,18:27:24,34.94,M002,Withdrawal,Canberra,Mobile,-37.020100,144.964600,emilyreese@gmail.com,False,104,1,AUD,GBP,2023-03-05 18:27:24,AUD
2,1145,2971,25,Male,Unemployed,Master,Married,2,74728.57,55873.76,1149.85,2023-11-10,17:16:56,323.82,M008,Purchase,Brisbane,Mobile,-31.840233,145.612793,fordevan@gmail.com,False,105,0,AUD,GBP,2023-11-10 17:16:56,AUD
3,15308,2925,25,Male,Professional,High School,Married,3,55712.62,89649.04,4335.70,2023-10-07,00:34:17,32.49,M001,Purchase,Darwin,Mobile,-37.020100,144.964600,kathleenlewis@tempmail.com,False,70,1,AED,AUD,2023-10-07 00:34:17,AED
4,14967,2339,38,Male,Professional,High School,Single,4,53004.70,43601.02,4763.48,2023-09-22,18:40:08,1140.75,M001,Withdrawal,Melbourne,Tablet,-37.020100,144.964600,kristinawhite@gmail.com,False,27,0,AED,AUD,2023-09-22 18:40:08,AED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,14541,106,47,Male,Student,Master,Married,4,74835.82,45101.32,1270.87,2023-04-26,12:28:58,477.47,M004,Payment,Brisbane,Mobile,-37.020100,144.964600,melaniereyes@gmail.com,True,67,0,AUD,GBP,2023-04-26 12:28:58,AUD
3564,12206,1668,45,Female,Unemployed,Master,Married,0,86312.26,48357.47,2009.35,2023-06-01,02:57:48,76.53,M007,Purchase,Perth,Desktop,-20.917574,142.702789,anthony35@gmail.com,False,63,1,AUD,GBP,2023-06-01 02:57:48,AUD
3565,8433,2701,30,Female,Student,High School,Single,3,50836.99,29800.75,1291.68,2023-07-23,13:37:05,9.88,M008,Payment,Canberra,Desktop,-37.020100,144.964600,mdixon@outlook.com,False,12,0,AUD,GBP,2023-07-23 13:37:05,AUD
3566,769,607,39,Male,Professional,High School,Single,1,64400.78,30010.69,203.90,2023-11-07,08:19:56,1434.47,M007,Purchase,Adelaide,Mobile,-37.020100,144.964600,jasminecastillo@outlook.com,False,52,0,AUD,GBP,2023-11-07 08:19:56,AUD


In [178]:
# One-hot encoding of categorical variables for train

cols = ['Income', 'Expenditure', 'GiftsTransaction', 'TransactionAmount', 'Occupation', 'TransactionType', 'TransactionLocation', 'Latitude', 'Longitude', 'Terrorism', 'ExpenditureCurrency', 'GiftsCurrency', 'TransactionCurrency', 'IsFraud']

dfi = df[cols]
dfi.head()

from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse_output = False)

oh_columns = ['Occupation', 'TransactionType', 'TransactionLocation', 'ExpenditureCurrency', 'GiftsCurrency', 'TransactionCurrency']

encoded = onehot.fit_transform(dfi[oh_columns])
encoded_cols = onehot.get_feature_names_out(oh_columns)
df_encoded = pd.DataFrame(encoded, columns = encoded_cols)
df_train = pd.concat([dfi.drop(oh_columns, axis=1), df_encoded], axis=1)
df_train['Terrorism'] = df_train['Terrorism'].astype(int)
df_train

from sklearn.model_selection import train_test_split

df_x = df_train.drop('IsFraud', axis=1)
df_y = df_train['IsFraud']

X_train, X_val, y_train, y_val = train_test_split(df_x, df_y, test_size = 0.2, random_state = 69)

In [179]:
# XGBoost

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, accuracy_score, make_scorer

# Create the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the hyperparameters to search over
param_dist = {
    'n_estimators': [100, 200, 300],          # Number of boosting rounds MIN
    'learning_rate': [0.01, 0.1, 0.2, 0.3],   # Learning rate 
    'max_depth': [4, 5, 6, 7],                # Maximum tree depth 
    'min_child_weight': [1, 3, 5],            # Minimum sum of instance weight needed in a child MIN
    'subsample': [0.6, 0.8, 1.0],             # Subsample ratio of the training instance MIN 
    'colsample_bytree': [0.6, 0.8, 1.0],      # Subsample ratio of columns when constructing each tree MIN
    'gamma': [0, 0.1, 0.2, 0.3],              # Minimum loss reduction required to make a further partition MAX
    'reg_alpha': [0, 0.01, 0.1, 1],           # L1 regularization term MAX
    'reg_lambda': [1, 0.1, 0.01],             # L2 regularization term
}

# Define the scorer to maximize F1 score
f1_scorer = make_scorer(f1_score)

# Set up the RandomizedSearchCV to search over the hyperparameters
random_search = RandomizedSearchCV(
    estimator=xgb_clf, 
    param_distributions=param_dist, 
    scoring=f1_scorer, 
    n_iter=100,            # Number of different parameter combinations to try
    cv=3,                 # 3-fold cross-validation
    verbose=2, 
    n_jobs=-1,            # Use all available cores
    random_state=69
)

# Fit the RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

# Best hyperparameters from RandomizedSearchCV
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate the model on the validation set
y_pred_val = random_search.predict(X_val)

# Calculate F1 score on the validation data
f1_val = f1_score(y_val, y_pred_val)
print("F1 score on validation data:", f1_val)

acc_score = accuracy_score(y_val, y_pred_val)
print("Accuracy score on validation data:", acc_score)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best Hyperparameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0.01, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.3, 'gamma': 0.3, 'colsample_bytree': 0.8}
F1 score on validation data: 0.6936936936936937
Accuracy score on validation data: 0.8095238095238095


c:\Users\pogiz\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:44:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [180]:
# One-hot encoding of categorical variables for test

test = pd.read_csv('data_clean/test.csv')

cols = ['Income', 'Expenditure', 'GiftsTransaction', 'TransactionAmount', 'Occupation', 'TransactionType', 'TransactionLocation', 'Latitude', 'Longitude', 'Terrorism', 'ExpenditureCurrency', 'GiftsCurrency', 'TransactionCurrency']

dft = test[cols]
dft.head()

from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse_output = False)

oh_columns = ['Occupation', 'TransactionType', 'TransactionLocation', 'ExpenditureCurrency', 'GiftsCurrency', 'TransactionCurrency']

encoded = onehot.fit_transform(dft[oh_columns])
encoded_cols = onehot.get_feature_names_out(oh_columns)
df_encoded = pd.DataFrame(encoded, columns = encoded_cols)
df_test = pd.concat([dft.drop(oh_columns, axis=1), df_encoded], axis=1)
df_test['Terrorism'] = df_test['Terrorism'].astype(int)
df_test


,Income,Expenditure,GiftsTransaction,TransactionAmount,Latitude,Longitude,Terrorism,Occupation_Professional,Occupation_Retired,Occupation_Student,Occupation_Unemployed,TransactionType_Payment,TransactionType_Purchase,TransactionType_Transfer,TransactionType_Withdrawal,TransactionLocation_Adelaide,TransactionLocation_Brisbane,TransactionLocation_Canberra,TransactionLocation_Darwin,TransactionLocation_Hobart,TransactionLocation_Melbourne,TransactionLocation_Perth,TransactionLocation_Sydney,ExpenditureCurrency_AED,ExpenditureCurrency_AUD,GiftsCurrency_AUD,GiftsCurrency_GBP,TransactionCurrency_AED,TransactionCurrency_AUD
0,53733.41,29296.02,2704.09,225.64,-37.020100,144.964600,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,54856.77,34628.31,959.96,658.10,-37.020100,144.964600,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,59011.72,33312.46,1964.20,133.59,-30.000233,136.209152,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,128795.40,67049.00,9516.18,6.74,-37.020100,144.964600,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,44506.03,22856.31,1737.75,15.67,-20.917574,142.702789,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,81116.67,42304.79,588.28,116.70,-31.840233,145.612793,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3045,38080.08,54722.46,4823.39,116.24,-37.020100,144.964600,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3046,102098.54,36394.21,6599.59,22.09,-37.020100,144.964600,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3047,64031.35,39187.08,2022.91,144.80,-20.917574,142.702789,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [181]:
test_pred = y_pred_val = random_search.predict(df_test)

In [182]:
tran_nums = test['TransactionNumber']
tran_nums

0       11854
1        2647
2        5945
3        6798
4       12985
        ...  
3044     8139
3045    15874
3046     6234
3047     4292
3048      296
Name: TransactionNumber, Length: 3049, dtype: int64

In [183]:
preds = pd.DataFrame(test_pred)
preds

,0
0,0
1,0
2,0
3,1
4,1
...,...
3044,0
3045,0
3046,0
3047,0


In [184]:
result = pd.DataFrame({
    'TransactionNumber': tran_nums,
    'IsFraud': preds[0]
})


In [185]:
result

,TransactionNumber,IsFraud
0,11854,0
1,2647,0
2,5945,0
3,6798,1
4,12985,1
...,...,...
3044,8139,0
3045,15874,0
3046,6234,0
3047,4292,0


In [186]:
result.to_csv('prediction.csv', index=False)